# Ultimate Exploratory Data Analysis (EDA)

This notebook contains a massive, deep-dive EDA into the Chest X-ray dataset.
It is divided into two main sections:
1.  **Metadata Analysis (12 Blocks)**: Analyzing patient demographics, disease co-occurrences, and distributions.
2.  **Image Analysis (12 Blocks)**: Analyzing pixel statistics, textures, frequency domains, and visual patterns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from PIL import Image
from glob import glob
from itertools import chain, combinations
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
from sklearn.decomposition import PCA

# Set style
plt.style.use('ggplot')
sns.set_palette("husl")

# Paths
BASE_DIR = '.'
CSV_PATH = os.path.join(BASE_DIR, 'Data_Entry_2017_v2020.csv')
IMAGE_DIRS = [
    os.path.join(BASE_DIR, 'images'),
    os.path.join(BASE_DIR, 'images 2')
]

## 0. Data Loading & Preprocessing

In [ ]:
df = pd.read_csv(CSV_PATH)

# Map images to paths
image_paths = {}
for img_dir in IMAGE_DIRS:
    if os.path.exists(img_dir):
        for fname in os.listdir(img_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths[fname] = os.path.join(img_dir, fname)

df['path'] = df['Image Index'].map(image_paths)
df = df.dropna(subset=['path'])

# Clean Age
df['Patient Age'] = pd.to_numeric(df['Patient Age'], errors='coerce')
df = df[df['Patient Age'] < 100]

# Process Labels
df['Label_List'] = df['Finding Labels'].str.split('|')
all_labels = list(chain(*df['Label_List'].tolist()))
unique_labels = sorted(list(set(all_labels)))

# One-Hot Encoding
mlb = MultiLabelBinarizer()
df_encoded = pd.concat([df, pd.DataFrame(mlb.fit_transform(df['Label_List']), columns=mlb.classes_, index=df.index)], axis=1)

print(f"Total images: {len(df)}")

# PART 1: METADATA ANALYSIS (12 Blocks)
This section focuses on the tabular data associated with each X-ray, providing insights into patient demographics and disease statistics.

### 1. Overall Disease Distribution
**Explanation:** This bar chart shows the total count of each disease label in the dataset. It helps identify class imbalance, showing which diseases are common (e.g., Infiltration) and which are rare (e.g., Hernia).

In [ ]:
label_counts = pd.Series(all_labels).value_counts()
plt.figure(figsize=(12,6))
label_counts.plot(kind='bar', color='cornflowerblue')
plt.title("1. Overall Disease Label Distribution")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')
plt.show()

### 2. Disease Co-occurrence Heatmap
**Explanation:** This heatmap visualizes how often two diseases appear together in the same patient. Darker squares indicate a higher frequency of co-occurrence, revealing clinical correlations (e.g., Effusion and Infiltration often occurring together).

In [ ]:
co_matrix = np.dot(df_encoded[unique_labels].T, df_encoded[unique_labels])
plt.figure(figsize=(12,10))
sns.heatmap(co_matrix, xticklabels=unique_labels, yticklabels=unique_labels, cmap="Blues", annot=True, fmt='d')
plt.title("2. Disease Co-occurrence Heatmap")
plt.show()

### 3. Patient Age Distribution
**Explanation:** A histogram showing the distribution of patient ages. This helps us understand the demographic profile of the dataset, such as whether it skews towards older or younger patients.

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df['Patient Age'], bins=20, kde=True, color='purple')
plt.title("3. Patient Age Distribution")
plt.show()

### 4. Gender Distribution
**Explanation:** A pie chart displaying the ratio of male to female patients. This is important for checking potential biases in the dataset.

In [ ]:
plt.figure(figsize=(6,6))
df['Patient Sex'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['lightblue', 'pink'])
plt.title("4. Gender Distribution")
plt.ylabel('')
plt.show()

### 5. Images per Patient (Repeat Scans)
**Explanation:** A histogram showing how many images exist per patient ID. Many patients have multiple follow-up scans, which is crucial to know for splitting data (to avoid data leakage between train/test sets).

In [ ]:
plt.figure(figsize=(10,5))
df['Patient ID'].value_counts().hist(bins=30, color='orange')
plt.title("5. Images per Patient (Repeat Scans)")
plt.xlabel("Number of Images")
plt.ylabel("Count of Patients")
plt.show()

### 6. Disease Count per Image (Comorbidity)
**Explanation:** This plot shows how many concurrent diseases are present in a single image. It highlights the complexity of the classification task (e.g., many images have 0 findings, while some have 3+).

In [ ]:
df['disease_count'] = df['Label_List'].apply(lambda x: len(x) if x != ['No Finding'] else 0)
plt.figure(figsize=(8,5))
sns.countplot(x='disease_count', data=df, palette='viridis')
plt.title("6. Number of Concurrent Diseases per Image")
plt.xlabel("Count of Diseases")
plt.show()

### 7. View Position Distribution
**Explanation:** Compares the count of PA (Posterior-Anterior) vs AP (Anterior-Posterior) X-ray views. PA is the standard view, while AP is often used for bedridden patients, which can affect image quality and heart size appearance.

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(x='View Position', data=df, palette='pastel')
plt.title("7. View Position Distribution (PA vs AP)")
plt.show()

### 8. Disease Frequency by View Position
**Explanation:** Analyzes if certain diseases are more common in specific views. For example, AP views might have more critical conditions since they are often taken for patients who cannot stand up.

In [ ]:
view_counts = df_encoded.groupby('View Position')[unique_labels].sum().T
view_counts.plot(kind='bar', figsize=(14,6))
plt.title("8. Disease Frequency by View Position")
plt.ylabel("Count")
plt.show()

### 9. Top 10 Common Disease Pairs
**Explanation:** Identifies the specific pairs of diseases that occur together most frequently. This is a more targeted version of the co-occurrence heatmap.

In [ ]:
pairs = []
for labels in df['Label_List']:
    if len(labels) > 1:
        pairs.extend(list(combinations(sorted(labels), 2)))
pair_counts = Counter(pairs).most_common(10)
labels = [f"{p[0]} + {p[1]}" for p, c in pair_counts]
counts = [c for p, c in pair_counts]

plt.figure(figsize=(12,6))
sns.barplot(x=counts, y=labels, palette='magma')
plt.title("9. Top 10 Disease Co-occurrence Pairs")
plt.show()

### 10. Disease Prevalence by Gender
**Explanation:** Compares the counts of the top 5 diseases between males and females to see if any condition disproportionately affects one gender.

In [ ]:
df_exploded = df.assign(Label=df['Finding Labels'].str.split('|')).explode('Label')
gender_disease = df_exploded.groupby(['Patient Sex', 'Label']).size().unstack(fill_value=0)
top_5_diseases = label_counts.head(5).index
gender_disease[top_5_diseases].plot(kind='bar', figsize=(10,6))
plt.title("10. Top 5 Diseases by Gender")
plt.ylabel("Count")
plt.show()

### 11. 'No Finding' Rate by Age Group
**Explanation:** A line chart showing the probability of a scan being "Normal" (No Finding) across different age groups. Typically, younger patients have higher rates of normal scans.

In [ ]:
df['age_bin'] = pd.cut(df['Patient Age'], bins=[0,20,40,60,80,100], labels=['0-20', '20-40', '40-60', '60-80', '80+'])
df['Is_Healthy'] = df['Finding Labels'] == 'No Finding'
healthy_rate = df.groupby('age_bin')['Is_Healthy'].mean()

plt.figure(figsize=(10,5))
healthy_rate.plot(kind='line', marker='o', color='green', linewidth=2)
plt.title("11. Probability of 'No Finding' by Age Group")
plt.ylabel("Probability")
plt.grid(True)
plt.show()

### 12. Correlation of Numeric Metadata
**Explanation:** A correlation matrix for numeric fields (Age, Follow-up #, etc.). It helps identify linear relationships, such as whether older patients tend to have more follow-up scans.

In [ ]:
numeric_df = df.select_dtypes(include=[np.number])
plt.figure(figsize=(10,8))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("12. Correlation Matrix of Numeric Metadata")
plt.show()

# PART 2: IMAGE ANALYSIS (12 Blocks)
This section applies computer vision techniques to analyze the actual pixel data of the X-rays.

### 13. Sample Image Grid
**Explanation:** Displays a random selection of 9 images from the dataset along with their labels. This provides a qualitative "sanity check" of the data quality and variety.

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(12,12))
samples = df.sample(9)
for i, (idx, row) in enumerate(samples.iterrows()):
    img = Image.open(row['path']).convert('L')
    ax[i//3][i%3].imshow(img, cmap='gray')
    ax[i//3][i%3].set_title(row['Finding Labels'], fontsize=8)
    ax[i//3][i%3].axis('off')
plt.suptitle("13. Random Sample Images")
plt.show()

### 14. Eigen-Chests (PCA Analysis)
**Explanation:** Uses Principal Component Analysis (PCA) to find the "Eigen-images" or principal components. These ghostly images represent the most common patterns of variance in the dataset (e.g., the overall shape of the ribcage).

In [ ]:
def compute_pca(paths, n_components=5, size=(64, 64)):
    images = []
    for p in paths:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, size)
            images.append(img.flatten())
    pca = PCA(n_components=n_components)
    pca.fit(np.array(images))
    return pca, size

sample_paths = df['path'].sample(min(300, len(df))).tolist()
pca, img_size = compute_pca(sample_paths)

plt.figure(figsize=(15, 4))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(pca.components_[i].reshape(img_size), cmap='gray')
    plt.title(f'PC #{i+1}')
    plt.axis('off')
plt.suptitle("14. Eigen-Chests (Principal Components)")
plt.show()

### 15. FFT Frequency Analysis
**Explanation:** Applies a Fast Fourier Transform (FFT) to an image. The resulting spectrum shows the frequency content: center is low frequency (smooth areas), edges are high frequency (sharp details/noise). This helps assess image sharpness/noise.

In [ ]:
img_path = df['path'].sample(1).values[0]
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
magnitude_spectrum = 20*np.log(np.abs(fshift))

plt.figure(figsize=(10,5))
plt.subplot(121), plt.imshow(img, cmap='gray'), plt.title('Original'), plt.axis('off')
plt.subplot(122), plt.imshow(magnitude_spectrum, cmap='gray'), plt.title('FFT Spectrum'), plt.axis('off')
plt.suptitle("15. Frequency Domain Analysis")
plt.show()

### 16. Edge Detection (Sobel)
**Explanation:** Uses Sobel filters to detect horizontal and vertical edges. This simulates how a Convolutional Neural Network (CNN) sees the image in its early layers, identifying boundaries of ribs, heart, and diaphragm.

In [ ]:
sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)

plt.figure(figsize=(12,4))
plt.subplot(131), plt.imshow(img, cmap='gray'), plt.title('Original'), plt.axis('off')
plt.subplot(132), plt.imshow(sobelx, cmap='gray'), plt.title('Sobel X (Vertical)'), plt.axis('off')
plt.subplot(133), plt.imshow(sobely, cmap='gray'), plt.title('Sobel Y (Horizontal)'), plt.axis('off')
plt.suptitle("16. Edge Detection")
plt.show()

### 17. Histogram Equalization
**Explanation:** Demonstrates Histogram Equalization, a technique to improve image contrast by spreading out pixel intensities. This is often used as a preprocessing step to make subtle features more visible.

In [ ]:
equ = cv2.equalizeHist(img)
res = np.hstack((img, equ))
plt.figure(figsize=(12,6))
plt.imshow(res, cmap='gray')
plt.title("17. Original vs. Histogram Equalized")
plt.axis('off')
plt.show()

### 18. Average Image Comparison
**Explanation:** Computes the "mean image" for two classes: Cardiomegaly and Normal. By averaging hundreds of images, noise cancels out, revealing the "average" structure. You should see a larger heart shadow in the Cardiomegaly average.

In [ ]:
def get_avg_img(paths, size=(256, 256)):
    avg_img = np.zeros(size)
    count = 0
    for p in paths:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, size)
            avg_img += img
            count += 1
    return avg_img / count if count > 0 else None

cardio_paths = df[df['Finding Labels'].str.contains('Cardiomegaly')]['path'].sample(min(100, len(df))).tolist()
normal_paths = df[df['Finding Labels'] == 'No Finding']['path'].sample(min(100, len(df))).tolist()

avg_cardio = get_avg_img(cardio_paths)
avg_normal = get_avg_img(normal_paths)

fig, ax = plt.subplots(1, 2, figsize=(12,6))
if avg_cardio is not None:
    ax[0].imshow(avg_cardio, cmap='gray')
    ax[0].set_title("Avg Cardiomegaly")
    ax[0].axis('off')
if avg_normal is not None:
    ax[1].imshow(avg_normal, cmap='gray')
    ax[1].set_title("Avg Normal")
    ax[1].axis('off')
plt.suptitle("18. Average Image Comparison")
plt.show()

### 19. Pixel Intensity Distribution
**Explanation:** Compares the distribution of pixel values for Effusion (fluid accumulation, appears white/bright) vs. Pneumothorax (air accumulation, appears black/dark). We expect Effusion to have a distribution shifted towards higher pixel values.

In [ ]:
def get_intensities(paths):
    vals = []
    for p in paths:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is not None: vals.append(np.mean(img))
    return vals

eff_vals = get_intensities(df[df['Finding Labels'].str.contains('Effusion')]['path'].sample(min(100, len(df))).tolist())
pneu_vals = get_intensities(df[df['Finding Labels'].str.contains('Pneumothorax')]['path'].sample(min(100, len(df))).tolist())

plt.figure(figsize=(10,6))
sns.kdeplot(eff_vals, label='Effusion (Fluid)', shade=True, color='blue')
sns.kdeplot(pneu_vals, label='Pneumothorax (Air)', shade=True, color='red')
plt.title("19. Pixel Intensity Distribution: Fluid vs Air")
plt.legend()
plt.show()

### 20. Image Aspect Ratio Distribution
**Explanation:** A histogram of image aspect ratios (Width / Height). Most X-rays are square (1.0) or slightly rectangular. Outliers might indicate cropped or rotated images.

In [ ]:
def get_ar(path):
    w, h = Image.open(path).size
    return w/h
ars = [get_ar(p) for p in df['path'].sample(min(500, len(df)))]
plt.figure(figsize=(8,5))
sns.histplot(ars, bins=20, kde=True, color='teal')
plt.title("20. Image Aspect Ratio Distribution")
plt.show()

### 21. Overall Image Brightness Distribution
**Explanation:** Shows the distribution of average brightness across the dataset. This helps identify if the dataset has significant lighting variations that might require normalization.

In [ ]:
brightness = [np.mean(Image.open(p).convert('L')) for p in df['path'].sample(min(500, len(df)))]
plt.figure(figsize=(10,5))
sns.histplot(brightness, bins=30, color='gold', kde=True)
plt.title("21. Overall Image Brightness Distribution")
plt.show()

### 22. Overall Image Contrast (Std Dev) Distribution
**Explanation:** Shows the distribution of image contrast (measured by the standard deviation of pixel intensities). Low contrast images might be harder for models to learn from.

In [ ]:
contrast = [np.std(Image.open(p).convert('L')) for p in df['path'].sample(min(500, len(df)))]
plt.figure(figsize=(10,5))
sns.histplot(contrast, bins=30, color='crimson', kde=True)
plt.title("22. Overall Image Contrast (Std Dev) Distribution")
plt.show()

### 23. Image Sharpness (Laplacian Variance)
**Explanation:** Uses the variance of the Laplacian filter as a proxy for image sharpness. Higher values mean sharper edges, while lower values indicate blurriness. This is useful for quality control.

In [ ]:
def get_sharpness(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return cv2.Laplacian(img, cv2.CV_64F).var()

sharpness = [get_sharpness(p) for p in df['path'].sample(min(300, len(df)))]
plt.figure(figsize=(10,5))
sns.histplot(sharpness, bins=30, color='green', kde=True)
plt.title("23. Image Sharpness Distribution (Laplacian Variance)")
plt.show()

### 24. Brightness Distribution by Top 5 Diseases
**Explanation:** A boxplot comparing the average brightness of images for the top 5 most common diseases. This checks if the model could potentially cheat by just looking at overall image brightness (e.g., if all Pneumonia images were darker).

In [ ]:
def get_avg_brightness(path):
    return np.mean(Image.open(path).convert('L'))

df_sample = df.sample(min(500, len(df)))
df_sample['brightness'] = df_sample['path'].apply(get_avg_brightness)
df_sample['Main_Label'] = df_sample['Label_List'].apply(lambda x: x[0] if len(x)>0 else 'Unknown')

top_labels = df_sample['Main_Label'].value_counts().head(5).index
df_top = df_sample[df_sample['Main_Label'].isin(top_labels)]

plt.figure(figsize=(12,6))
sns.boxplot(x='Main_Label', y='brightness', data=df_top)
plt.title("24. Brightness Distribution by Top 5 Diseases")
plt.show()